# **Demo: MultiPDF QA Retriever with ChromaDB and LangChain**

In this demo, you will learn how to use LangChain to create a MultiPDF retriever with ChromaDB. This demo is performed on new generative AI research paper PDFs. You will understand how to load and process documents, create a database, make a retriever, create a chain, and use the retriever to ask questions and get answers.

## **Steps to Perform:**

*   Step 1: Importing the Necessary Libraries
*   Step 2: Loading and Splitting
*   Step 3: Loading the OpenAI Embeddings
*   Step 4: Creating and Loading the Database
*   Step 5: Creating and Using the Retriever
*   Step 6: Passing the Query



### **Step 1: Importing the Necessary Libraries**

In [ ]:
# !pip install langchain-community
# !pip install pypdf
# !pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 10.2 MB/s et

In [ ]:
from google.colab import userdata
import os

api_key = userdata.get('secret_key')

os.environ["OPENAI_API_KEY"] = api_key
# openai.api_key = api_key

In [ ]:
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader, PyPDFLoader, DirectoryLoader
from langchain.embeddings import OpenAIEmbeddings
import os
import openai


### **Step 2: Loading and Splitting**


*   Create a directory named `GenAI_Papers`.
*   Load the PDF documents in the directory.
*   Split the documents into smaller chunks using the **RecursiveCharacterTextSplitter**.

In [ ]:
from google.colab import drive
# drive.mount('/content/drive')

folder_path = '/content/drive/MyDrive/GenAI_Papers'

In [ ]:
#Loading the documents
# doc_loader = DirectoryLoader('GenAI_Papers', glob="./*.pdf", loader_cls=PyPDFLoader)
doc_loader = DirectoryLoader(folder_path, glob="./*.pdf", loader_cls=PyPDFLoader)
documents = doc_loader.load()

#Splitting the documents
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)


In [ ]:
# texts[1].page_content

'complex task into simpler subtasks while the latter consist s in calling external modules such\nas a code interpreter. LMs can leverage these augmentations separately or in combination\nvia heuristics, or learn to do so from demonstrations. While adhering to a standard missing\ntokens prediction objective, such augmented LMs can use var ious, possibly non-parametric\nexternal modules to expand their context processing abilit y , thus departing from the pure\nlanguage modeling paradigm. W e therefore refer to them as Au gmented Language Models\n(ALMs). The missing token objective allows ALMs to learn to r eason, use tools, and even\nact, while still performing standard natural language task s and even outperforming most\nregular LMs on several benchmarks. In this work, after revie wing current advance in ALMs,\nwe conclude that this new research direction has the potenti al to address common limitations\nof traditional LMs such as interpretability , consistency , and scalability issues

### **Step 3: Loading the OpenAI Embeddings**

In [ ]:
openai_embeddings = OpenAIEmbeddings()

### **Step 4: Creating and Loading the Database**

*   Create a database to store the embedded text.
*   Load the database to bring it back into memory from the disk.



In [ ]:
#Creating a database
persist_directory = 'db'
embedding = openai_embeddings

# vector databse from our docs (embedded)
vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embedding,
                                 persist_directory=persist_directory)
vectordb.persist()
vectordb = None


#Loading the database
vectordb = Chroma(persist_directory=persist_directory,
                  embedding_function=embedding)


/tmp/ipython-input-27-3152848975.py:7: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()
/tmp/ipython-input-27-3152848975.py:10: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(persist_directory=persist_directory,


### **Step 5: Creating and Using the Retriever**

*   Create a retriever using the vector database.
*   Use the retriever to get relevant documents for a specific query.



In [ ]:
#Creating retriever
retriever = vectordb.as_retriever()

#Using retriever
docs = retriever.get_relevant_documents("What is toolformer?")


/tmp/ipython-input-28-1482675577.py:5: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents("What is toolformer?")


In [ ]:
# docs[1].page_content

'cently ChatGPT 2. Memorization ( Tirumala et al. , 2022) combined with compositionality ( Zhou et al. , 2022)\ncapabilities made LLMs able to execute various tasks such as language understanding or conditional and un-\nconditional text generation at an unprecedented level of pe rformance, thus opening a realistic path towards\nhigher-bandwidth human-computer interactions.\nHowever, LLMs suﬀer from important limitations hindering a broader deployment. LLMs often provide non-\nfactual but seemingly plausible predictions, often referr ed to as hallucinations (\nW elleck et al. , 2020). This\nleads to many avoidable mistakes, for example in the context of arithmetics ( Qian et al. , 2022) or within\na reasoning chain ( W ei et al. , 2022c). Moreover, many LLMs groundbreaking capabilities seem to emerge\nwith size, measured by the number of trainable parameters: f or example, W ei et al. (2022b) demonstrate'

### **Step 6: Passing the Query**

*   Pass the query to the vector database.
*   Print the content of the most relevant document.



In [ ]:
query = "A fundamental limitation of HMMs"
docs = vectordb.similarity_search(query)
print(docs[0].page_content)


with size, measured by the number of trainable parameters: f or example, W ei et al. (2022b) demonstrate
that LLMs become able to perform some BIG-bench tasks 3 via few-shot prompting once a certain scale is
attained. Although a recent line of work yielded smaller LMs that retain some capabilities from their largest
counterpart ( Hoﬀmann et al. , 2022), the size and need for data of LLMs can be impractical for tra ining
but also maintenance: continual learning for large models r emains an open research question ( Scialom et al. ,
2022). Other limitations of LLMs are discussed by Goldberg (2023) in the context of ChatGPT, a chatbot
built upon GPT3.
W e argue these issues stem from a fundamental defect of LLMs: they are generally trained to perform
statistical language modeling given (i) a single parametri c model and (ii) a limited context, typically the n
previous or surrounding tokens. While n has been growing in recent years thanks to software and hardw are


### **Conclusion**

By the end of this demo, you have a clear understanding of how to use LangChain’s MultiPDF retriever with ChromaDB. You’ve learned how to load and process documents, create a database, make a retriever, and use the retriever to ask questions. This knowledge will help you effectively utilize LangChain’s capabilities in your projects.